In [1]:
from datetime import datetime, timedelta
import pandas as pd

In [2]:
# --- 関数定義：時刻表処理と集計 ---
def parse_starred_time_text(text):
    lines = text.strip().splitlines()
    current_hour = None
    times = []
    for line in lines:
        line = line.strip()
        if not line:
            continue
        if line.startswith("*"):
            part = ''.join(filter(str.isdigit, line))
            if current_hour and part:
                times.append(f"{current_hour}:{part.zfill(2)}")
        elif line.isdigit():
            current_hour = line.zfill(2)
    return [datetime.strptime(t, "%H:%M").time() for t in times]

def make_time_bins(start="06:00", end="23:59"):
    bins, labels = [], []
    start_dt = datetime.strptime(start, "%H:%M")
    end_dt = datetime.strptime(end, "%H:%M")
    while start_dt < end_dt:
        bin_start = start_dt.time()
        bin_end = (start_dt + timedelta(minutes=14)).time()
        label = f"{bin_start.strftime('%H:%M')}-{bin_end.strftime('%H:%M')}"
        bins.append((bin_start, bin_end))
        labels.append(label)
        start_dt += timedelta(minutes=15)
    return bins, labels

def count_by_time_bins(time_list, bins, labels):
    count_dict = {label: 0 for label in labels}
    for t in time_list:
        for (bin_start, bin_end), label in zip(bins, labels):
            if bin_start <= t <= bin_end:
                count_dict[label] += 1
                break
    return count_dict

def make_frequency_df(direction_dict, route, station, source="都営バス"):
    bins, labels = make_time_bins()
    data_rows = []
    for dest, text in direction_dict.items():
        parsed_times = parse_starred_time_text(text)
        counts = count_by_time_bins(parsed_times, bins, labels)
        data_rows.append(list(counts.values()))
    index = pd.MultiIndex.from_product([[route], [station], list(direction_dict.keys())],
                                       names=["路線", "基準駅", "行き先"])
    df = pd.DataFrame(data_rows, index=index, columns=labels)
    df.insert(0, "出典", source)
    return df

In [ ]:
# --- 時刻表データを入力 ---
text_ueno = """06
* 		 18 
* 		 53 
07
* 		 09 
* 		 25 
* 		 46 
08
* 		 03 
* 		 19 
* 		 35 
* 		 53 
09
* 		 12 
* 		 36 
10
* 		 00 
* 		 24 
* 		 50 
11
* 		 16 
* 		 43 
12
* 		 10 
* 		 37 
13
* 		 05 
* 		 33 
14
* 		 01 
* 		 29 
* 		 55 
15
* 		 19 
* 		 43 
16
* 		 07 
* 		 31 
* 		 55 
17
* 		 19 
* 		 45 
18
* 		 10 
* 		 38 
19
* 		 02 
* 		 40 
20
* 		 29 
21
* 		 32
"""

text_hirai = """07
* 08
* 36
* 53
08
* 08
* 31
* 52"""

In [ ]:
# --- 各地点・路線・行き先のデータを辞書で定義 ---
all_data = {
    ("上23", "押上駅"): {
        "上野松坂屋前": text_ueno,
        "平井駅前": text_hirai
    }
    # 必要に応じて追加...
}

In [ ]:
# --- 各組み合わせごとに DataFrame を生成し、まとめて結合 ---
all_dfs = []
for (route, station), direction_dict in all_data.items():
    df = make_frequency_df(direction_dict, route=route, station=station)
    all_dfs.append(df)

# --- 結合して最終的な DataFrame を作成 ---
final_df = pd.concat(all_dfs)

In [ ]:
# --- Excelファイルとして保存 ---
final_df.to_csv("bus_all.csv")